In [1]:
import numpy as np
import pandas as pd
import face_recognition
import cv2
import os

In [2]:
raw = pd.read_csv('face_dataset/performance.csv')
raw.head()

,Face_ID,Performance,Video URL
0,1,1.1060,https://fgimagestorage.blob.core.windows.net/f...
1,2,1.3930,https://fgimagestorage.blob.core.windows.net/f...
2,2,0.6434,NaN
3,2,0.6250,NaN
4,2,0.9191,NaN


In [3]:
raw.shape

(2212, 3)

In [4]:
url_directory = raw[raw['Video URL'].notna()]
url_directory = url_directory[['Face_ID', 'Video URL']]
url_directory.head()

,Face_ID,Video URL
0,1,https://fgimagestorage.blob.core.windows.net/f...
1,2,https://fgimagestorage.blob.core.windows.net/f...
18,3,https://fgimagestorage.blob.core.windows.net/f...
30,4,https://fgimagestorage.blob.core.windows.net/f...
52,5,https://fgimagestorage.blob.core.windows.net/f...


In [5]:
ranking = raw[['Face_ID', 'Performance']].drop_duplicates()
ranking.reset_index(inplace=True, drop=True)
print(ranking.shape)
ranking.head()

(383, 2)


,Face_ID,Performance
0,1,1.1060
1,2,1.3930
2,2,0.6434
3,2,0.6250
4,2,0.9191


In [6]:
ranking = ranking.groupby('Face_ID', as_index=False)['Performance'].mean()
ranking.rename(columns={'Performance': 'Avg_Performance'}, inplace=True)
ranking.sort_values(by='Avg_Performance', ascending=False, inplace=True)
ranking.reset_index(inplace=True, drop=True)
ranking = pd.merge(ranking, url_directory, on='Face_ID')
ranking.head()

,Face_ID,Avg_Performance,Video URL
0,8,1.731514,https://fgimagestorage.blob.core.windows.net/f...
1,7,1.686180,https://fgimagestorage.blob.core.windows.net/f...
2,9,1.678143,https://fgimagestorage.blob.core.windows.net/f...
3,12,1.618300,https://fgimagestorage.blob.core.windows.net/f...
4,15,1.543925,https://fgimagestorage.blob.core.windows.net/f...


In [7]:
ranking.to_csv('Influencer_Ranking.csv', index=False)

In [ ]:
topface = ranking['Face_ID'][0]
file = 'face_dataset/face_{}.npy'.format(topface)
target_encoding = np.load(file, allow_pickle=True)
tolerance = 0.2

# Open the video file
url = url_directory[url_directory['Face_ID'] == topface]['Video URL'].values[0]
cap = cv2.VideoCapture(url)
print("Captured the video")

while cap.isOpened():
    print("Reading new frame")
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame from BGR (OpenCV format) to RGB (face_recognition format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        print("Comparing the encoding to the target face")
        matches = face_recognition.compare_faces([target_encoding], face_encoding, tolerance=tolerance)
        if matches[0]: 
            print("Match found!")
            top, right, bottom, left = face_location

            # Saving the face
            face_image = frame[top:bottom, left:right]
            cv2.imwrite("top_influencer.jpg", face_image)
            break

cap.release()